<a href="https://colab.research.google.com/github/EstSanchez/CVRP-Merida/blob/main/VRP_PPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120500 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [4]:
from pyomo.environ import *
from itertools import combinations
import time

start = time.time()

model = pyomo.environ.ConcreteModel()




numVeh = 3
capacidad = 12
numNodos = 10

costoNodos = [0, 3, 3, 3, 2, 4, 3, 4, 4, 1, 4]

costoAristas = [[50.79, 856.42, 1413.8, 363.11, 3037.11, 756.9, 2523.16, 4166.49, 4140.17, 1940.91, 2672.65],
[980.35, 103.16, 931.04, 597.12, 3718.78, 388.41, 3204.83, 4848.16, 4821.84, 1319.4, 3354.32],
[1622.33, 946.31, 93.31, 1401.41, 4002.56, 1024.51, 3721.5, 5131.94, 5320.76, 995.16, 3798.33],
[490.25, 707.55, 1357.02, 47.81, 3203.12, 608.03, 2689.17, 4332.5, 4306.18, 1758.01, 2838.66],
[3126.92, 3638.23, 3929.28, 3306.42, 126.41, 3538.71, 1114.53, 1706.26, 1848.95, 4485.78, 965.04],
[866.23, 375.34, 1024.81, 469.4, 3591.06, 92.5, 3077.11, 4720.44, 4694.12, 1425.8, 3226.6],
[2656.25, 3379.17, 3670.22, 2968.57, 860.43, 3279.65, 149.49, 2093.81, 2096.48, 4226.72, 298.98],
[4089.9, 4601.21, 4892.26, 4269.4, 1728.87, 4501.69, 2138.98, 84.5, 1181.86, 5448.76, 1989.49],
[4545.06, 5308.59, 5599.64, 4857.38, 2347.52, 5209.07, 2645.53, 1531.42, 21.03, 6156.14, 2496.04],
[2208.56, 1450.19, 1239.7, 1877.04, 4616.03, 1528.39, 4334.97, 5745.41, 5934.23, 167.57, 4411.8],
[2506.76, 3291.04, 3645.93, 2819.08, 1009.92, 3191.52, 298.98, 2243.3, 2245.97, 4202.43, 149.49]]






model = ConcreteModel()

model.nodos = RangeSet(numNodos+1)
model.nodosS = RangeSet(2, numNodos+1)
model.veh = RangeSet(numVeh)

model.x = Var(model.nodos, model.nodos, model.veh, domain = Binary)


model.obj = Objective(expr = sum(costoAristas[i-1][j-1] * model.x[i, j, k]
                                 for i in model.nodos
                                 for j in model.nodos
                                 for k in model.veh), sense = minimize)



#No existen rutas que vayan de un nodo a si mismo
model.restriccion_nolazos = ConstraintList()
for k in model.veh:
  for i in model.nodos:
    model.restriccion_nolazos.add(
        model.x[i, i, k] == 0
    )

#La suma de la demanda de los nodos en la ruta de un vehículo es menor o igual a la capacidad de este vehículo
model.restriccion_capacidad = ConstraintList()
for k in model.veh:
  model.restriccion_capacidad.add(
      sum(costoNodos[j-1] * model.x[i, j, k]
          for i in model.nodos
          for j in model.nodos) <= capacidad
  )

#El mismo vehículo sale del nodo que entra
model.restriccion_leavesNode = ConstraintList()
for k in model.veh:
  for j in model.nodos:
    model.restriccion_leavesNode.add(
        sum(model.x[i,j,k] for i in model.nodos) == sum(model.x[j,i,k] for i in model.nodos)
    )

#Cada nodo es pasado por exactamente 1 vehículo
model.restriccion_arrivesNode = ConstraintList()
for j in model.nodos:
    if j >= 2:
      model.restriccion_arrivesNode.add(
      sum(model.x[i,j,k] for i in model.nodos for k in model.veh if i != j) ==1
  )

#Todos los vehículos salen del almacen
model.restriccion_leavesDepot = ConstraintList()
for k in model.veh:
  model.restriccion_leavesDepot.add(
      sum(model.x[1, j, k] for j in model.nodos if j >= 2) == 1
  )


#Se eliminan las subrutas
model.restriccion_noSubtours = ConstraintList()
for r in range(2, numNodos):
  for s in combinations(model.nodosS, r):
    model.restriccion_noSubtours.add(
        sum(model.x[i, j, k] for i in s for j in s if j!=i for k in model.veh) <= len(s)-1
    )



solver = SolverFactory('glpk')
result = solver.solve(model)


if result.solver.termination_condition == TerminationCondition.optimal:
    for k in model.veh:
        for i in model.nodos:
            for j in model.nodos:
                if value(model.x[i, j, k]) > 0:
                    print(f'x[{i-1},{j-1},{k}] = {costoAristas[i-1][j-1]}')

    print('Costo Óptimo', value(model.obj))
else:
    print('No se encontró solución óptima')


end = time.time()

print("time: " + str(end - start))


x[0,4,1] = 3037.11
x[4,7,1] = 1706.26
x[7,8,1] = 1181.86
x[8,0,1] = 4545.06
x[0,3,2] = 363.11
x[3,6,2] = 2689.17
x[6,10,2] = 298.98
x[10,0,2] = 2506.76
x[0,2,3] = 1413.8
x[1,5,3] = 388.41
x[2,9,3] = 995.16
x[5,0,3] = 866.23
x[9,1,3] = 1450.19
Costo Óptimo 21442.1
time: 239.98547506332397
